# Generate `nbgitpuller` links for your JupyterHub

When users click an `nbgitpuller` link pointing to your JupyterHub,

1. They are asked to log in to the JupyterHub if they have not already
2. The git repository referred to in the nbgitpuller link is made up to date in their home directory (keeping local changes if there are merge conflicts)
3. They are shown the specific notebook / directory referred to in the nbgitpuller link.

This is a great way to distribute materials to students.

In [ ]:
from ipywidgets import interact
from urllib.parse import urlunparse, urlparse, urlencode, parse_qs
from IPython.display import Markdown

@interact
def make_nbgitpuller_link(hub_url='', repo_url='', branch='', filepath='', app=['notebook', 'lab']):
    """Generate an ipywidget form that creates an interact link."""
    
    # Don't do anything if we don't have a hub_url or repo_url
    if not (len(hub_url) > 0 and len(repo_url) > 0):
        return
    
    # Parse the query to its constituent parts
    scheme, netloc, path, params, query_str, fragment = urlparse(hub_url.strip())
    
    # nbgitpuller takes arguments via query parameters.
    # However, your hub might already require query parameters to function (for example, to pick a profile to launch in)
    # So we preserve the parameters we get & amend them to include the git-pull info we want
    query = parse_qs(query_str, keep_blank_values=True)
    
    
    # Make sure the path doesn't contain multiple slashes
    if not path.endswith('/'):
        path += '/'
    path += 'hub/user-redirect/git-pull'
    
    # Construct query parameters from 
    for name, val in [('repo', repo_url), ('branch', branch), ('subPath', filepath), ('app', app)]:
        if len(val) > 0:
            query[name] = val.strip().strip('/')
    
    url = urlunparse((scheme, netloc, path, params, urlencode(query, doseq=True), fragment))
    
    # FIXME: Have this return something instead of print so we can unit test
    print(url)